In [1]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from gensim import corpora,models
from collections import Counter

In [2]:
stopwords=['కూడా', 'ఉన్నారు', 'ఎవరైనా', 'ఎప్పుడు', 'ఎక్కడైనా', 'అందుబాటులో', 'ఒక ప్రక్కన', 'సంబంధం', 'మాత్రమే', 'అడగండి', 'గా', 'మధ్య', 'ప్రకారం', 'అనుమతించు', 'మెచ్చుకో', 'వద్ద', 'తగిన', 'అడ్డంగా', 'అడగడం', 'ఇప్పటికే', 'కనిపిస్తాయి', 'కాదు', 'మరియు', 'మరొక', 'అనుగుణంగా', 'అయితే', 'చేయగలిగింది', 'దాదాపు', 'గురించి', 'పై', 'వెంట', 'వేరుగా', 'చుట్టూ', 'దూరంగా', 'ఏ', 'నిజంగా', 'అనుమతిస్తుంది', 'అందరూ', 'ఎవరో ఒకరు', 'వ్యతిరేకంగా', 'ఏదైనా', 'తర్వాత', 'మళ్ళీ', 'ఏమైనప్పటికి', 'ఒక']

In [3]:
vowels=set("అ ఆ ఇ ఈ ఉ ఊ ఋ ౠ ఌ ౡ ఎ ఏ ఐ ఒ ఓ ఔ అం అః".split())

conso="క	ఖ	గ	ఘ	ఙ	చ	ఛ	జ	ఝ	ఞ	ట	ఠ	డ	ఢ	ణ	త	థ	ద	ధ	న	ప	ఫ	బ	భ	మ	య	ర	ల	వ	శ	ష	స	హ	ళ	క్ష	ఱ"

matra = "ఀ	ఁ	ం	ః	ఄ	ఽ	ా	ి	ీ	ు	ూ	ృ	ౄ	ె	ే	ై	ొ	ో	ౌ	్	ౕ	ౖ	ౢ	ౣ"

halantha='్'

In [4]:
consonants=set(conso.split("\t"))
matra = set(matra.split("\t"))

vowels = list(vowels)
consonants=list(consonants)
matra=list(matra)

alphabets = vowels+consonants

In [5]:
print(alphabets)

['ఋ', 'అ', 'ౡ', 'ఐ', 'ఒ', 'ఈ', 'ఓ', 'అం', 'ఇ', 'అః', 'ౠ', 'ఊ', 'ఌ', 'ఏ', 'ఔ', 'ఆ', 'ఉ', 'ఎ', 'శ', 'ఖ', 'థ', 'చ', 'హ', 'మ', 'ఞ', 'ద', 'ల', 'ళ', 'క్ష', 'డ', 'య', 'ఱ', 'గ', 'ర', 'బ', 'జ', 'ఝ', 'క', 'ఠ', 'న', 'ష', 'ఫ', 'ఘ', 'వ', 'ట', 'ఢ', 'ణ', 'ఙ', 'ఛ', 'ప', 'త', 'భ', 'స', 'ధ']


In [6]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())

In [7]:
data=pd.read_csv('../telugu.csv')
data

,Chapter,Verse,Sloke,Commentary
0,1,1,धृतराष्ट्र उवाच\n\nधर्मक्षेत्रे कुरुक्षेत्रे स...,ధృతరాష్ట్ర మహారాజు పుట్టుకతోనే గుడ్డి వాడే కాక...
1,1,2,सञ्जय उवाच\n\nदृष्ट्वा तु पाण्डवानीकं व्यूढं द...,తన పుత్రులు ఎలాగైనా యుద్ధం మొదలు పెడతారనే ధ్రు...
2,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम्।\n\...,తన అస్త్రవిద్యా గురువు ద్రోణాచార్యుడికి గతంలో ...
3,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि।\n\nयुयु...,"రాబోయే పెను విపత్తు భయానికి, పాండవులు సమీకరించ..."
4,1,5,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्।\n\nप...,"రాబోయే పెను విపత్తు భయానికి, పాండవులు సమీకరించ..."
...,...,...,...,...
689,18,74,सञ्जय उवाचइत्यहं वासुदेवस्य पार्थस्य च महात्मन...,"ఈ విధంగా సంజయుడు, భగవద్ గీత అనే దివ్య ఉపదేశమున..."
690,18,75,व्यासप्रसादाच्छ्रुतवानेतद्गुह्यमहं परम्।योगं य...,"శ్రీ కృష్ణ ద్వైపాయన వ్యాసదేవుడు, ఆయననే మహర్షి ..."
691,18,76,राजन्संस्मृत्य संस्मृत्य संवादमिममद्भुतम्।केशव...,ఆధ్యాత్మిక అనుభవము అనేది సమస్త భౌతిక ఆనందాలని ...
692,18,77,तच्च संस्मृत्य संस्मृत्य रूपमत्यद्भुतं हरेः।\n...,భగవంతుని విశ్వ రూప దర్శన భాగ్యము అర్జునుడికి క...


In [8]:
doc_words=[]
for index in range(len(data)):
    text=data.loc[index]['Commentary']
    text = remove_special_characters(text)
    words = text.split()
    words = [word for word in words if word not in stopwords]
    doc_words.append(words)

In [9]:
dictionary = corpora.Dictionary(doc_words)
corpus = [dictionary.doc2bow(text) for text in doc_words]

In [10]:
num_topics = 30
lsi=models.LsiModel(corpus,id2word=dictionary,num_topics=num_topics)

In [11]:
lsi.save('lsi.model')

In [12]:
dictionary.save('dictionary')
corpora.MmCorpus.serialize('corpus',corpus)